In [1]:
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# # !pip install -U PyDrive
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())


In [ ]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# gauth = GoogleAuth()
# # Try to load saved client credentials
# gauth.LoadCredentialsFile("mycreds.txt")
# if gauth.credentials is None:
#     # Authenticate if they're not there
#     gauth.LocalWebserverAuth()
# elif gauth.access_token_expired:
#     # Refresh them if expired
#     gauth.Refresh()
# else:
#     # Initialize the saved creds
#     gauth.Authorize()
# # Save the current credentials to a file
# gauth.SaveCredentialsFile("mycreds.txt")

# drive = GoogleDrive(gauth)


In [ ]:
#Import authentication libraries

# from google.colab import auth
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from oauth2client.client import GoogleCredentials

In [ ]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# folder_id='1uXkcRXuQ5fFG0S8mLW67ewj_jsqAb_Ff'

In [ ]:
# # Auto-iterate through all files in the midi folder.
# file_list = drive.ListFile({'q': "'{0}' in parents and trashed=false".format(folder_id)}).GetList()
# for file1 in file_list:
#   print('title: {0}, id: {1}'.format(file1['title'], file1['id']))

In [ ]:
# # Download all midi files
# for file1 in file_list:
#   file_id=file1['id']
#   file_title = file1['title']
#   drive.CreateFile({'id': file_id}).GetContentFile(file_title)
  

In [2]:
from os import listdir
from os.path import isfile, join

In [3]:
echo %cd%

C:\Users\Gustavo\Documents\GitHub\GAN_project_PUCP\Midinet_VG\Model_type1


In [4]:
folder ='C:/Users/Gustavo/Documents/GitHub/GAN_project_PUCP/Midinet_VG/Model_type1'


In [7]:
songs=[f for f in listdir(folder)]
songs=songs[15:-3]
songs

['song2_5tracks_103measures.pickle',
 'song3_1tracks_98measures.pickle',
 'song4_5tracks_59measures.pickle',
 'song5_5tracks_65measures.pickle',
 'song6_5tracks_72measures.pickle',
 'song7_5tracks_157measures.pickle',
 'song_1445_5tracks_34measures.pickle',
 'song_Battle1_5tracks_54measures.pickle',
 'song_Battlethemeremix_5tracks_38measures.pickle',
 'song_bkgr_freezing_furnace_5tracks_120measures.pickle',
 'song_Bricksterbots_4tracks_33measures.pickle',
 'song_CVAoS-DanceHall_5tracks_186measures.pickle',
 'song_CVAoS-TheArena_5tracks_201measures.pickle',
 'song_DE-AW2Opening_5tracks_19measures.pickle',
 'song_Donut_Plains_Remix_5tracks_100measures.pickle',
 'song_F-Zero_GP_Legend_-_Fire_Field_5tracks_98measures.pickle',
 'song_FF5_HOM2_5tracks_96measures.pickle',
 'song_FFGBAstage1_5tracks_53measures.pickle',
 'song_FZC_Silence-KM_5tracks_129measures.pickle',
 'song_gba-pkmnfrlg-profoakencounter_5tracks_42measures.pickle',
 'song_GS_MercuryLighthouse_5tracks_72measures.pickle',
 'son

In [8]:
data_songs=np.empty((0,16,128), int)
for song in songs:
    try:  
      song_opened = open(song,"rb")
      song_loaded= pickle.load(song_opened)
      #for track 1
      for i in range(len(song_loaded[0])):
        bar=np.empty((0,128), int)[None]
        for j in song_loaded[0][i]:
          note=np.zeros(128)[None][None]
          if type(j) == type(1.5):
            note[0,0,int(j)] = 1
          bar=np.append(bar,note,axis=1)
        data_songs =np.append(data_songs,bar, axis=0)
      print(np.array(data_songs).shape,'song=',song,np.array(song_loaded).shape)
    except:
        print('error parsing')
data_songs_prev =np.roll(data_songs,1,axis=0)
data_songs=data_songs[None]
data_songs_prev=data_songs_prev[None]

(103, 16, 128) song= song2_5tracks_103measures.pickle (5, 103, 16)
(201, 16, 128) song= song3_1tracks_98measures.pickle (1, 98, 16)
(260, 16, 128) song= song4_5tracks_59measures.pickle (5, 59, 16)
(325, 16, 128) song= song5_5tracks_65measures.pickle (5, 65, 16)
(397, 16, 128) song= song6_5tracks_72measures.pickle (5, 72, 16)
(554, 16, 128) song= song7_5tracks_157measures.pickle (5, 157, 16)
(588, 16, 128) song= song_1445_5tracks_34measures.pickle (5, 34, 16)
(642, 16, 128) song= song_Battle1_5tracks_54measures.pickle (5, 54, 16)
(680, 16, 128) song= song_Battlethemeremix_5tracks_38measures.pickle (5, 38, 16)
(800, 16, 128) song= song_bkgr_freezing_furnace_5tracks_120measures.pickle (5, 120, 16)
(833, 16, 128) song= song_Bricksterbots_4tracks_33measures.pickle (4, 33, 16)
(1019, 16, 128) song= song_CVAoS-DanceHall_5tracks_186measures.pickle (5, 186, 16)
(1220, 16, 128) song= song_CVAoS-TheArena_5tracks_201measures.pickle (5, 201, 16)
(1239, 16, 128) song= song_DE-AW2Opening_5tracks_19me

In [9]:
print(data_songs.shape)
print(data_songs_prev.shape)
data_songs=np.transpose(data_songs, (1,0,2,3))#Real data, input for model
data_songs_prev=np.transpose(data_songs_prev, (1,0,2,3))#Real data, 1 previous bar, input for model
print(data_songs.shape)
print(data_songs_prev.shape)

(1, 4950, 16, 128)
(1, 4950, 16, 128)
(4950, 1, 16, 128)
(4950, 1, 16, 128)


In [10]:
np.save('data_songs.npy', data_songs, allow_pickle=True, fix_imports=True)

In [33]:
data_test = np.load('data_songs.npy')
data_test.squeeze().reshape(4950,16*128)[0:500].shape
len(data_test)

4950

In [21]:
testo=data_test.squeeze().reshape(4950,16*128)[0:500]

In [26]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.cluster import SpectralClustering
from sklearn.manifold import SpectralEmbedding
from sklearn.preprocessing import normalize

def ssc_mps_modificado(X,smax,L,tol=None,alg_name='OMP',pmax=None):
	"""
  Esta es una versión modificada de la función 'ssc_mps', implementada por
  https://github.com/mitscha/ssc_mps_py/blob/master/ssc_mps.py
  
	Implements Sparse Subspace Clustering-Orthogonal Matching Pursuit (SSC-OMP) and 
	SSC-Matching Pursuit (SSC-MP)
	
	Parameters
	----------
	
	X: array, shape (n_samples, n_features)
		data matrix
	smax: int
		Maximum number of OMP/MP iterations
	L: int
		Number of clusters
	tol: float, optional
		Threshold on approximation quality
	alg_name: string, optional
		'OMP' (default) or 'MP'
	pmax:
		Maximum sparsity level for MP
	
	
	Note: 
	
	- Stopping behavior:
	  SSC-OMP: Stop after smax iterations if tol=None, stop when approximation quality
	           specified by tol is attained otherwise
	  SSC-MP:  Stop after smax iterations, or when approximation quality specified by tol
	           is attained, or when the sparsity level of the coefficient vector is pmax
	- See https://arxiv.org/abs/1612.03450 for a discussion of the stopping criteria
	
	"""
	
	
	XX = np.array(X).T
	XX = normalize(XX - np.outer(np.mean(XX, axis=1), np.ones(XX.shape[1])), axis=0)
    
	assert(len(XX.shape) == 2)
	
	m = XX.shape[0]
	N = XX.shape[1]
	
	
	alg = None
	if alg_name == 'MP':
		alg = MatchingPursuit(smax, pmax, tol)
	else:
		alg = OrthogonalMatchingPursuit(
			n_nonzero_coefs=smax, 
			tol=tol, 
			fit_intercept=False, 
			normalize=False)
	
	
	C = np.zeros((N,N))
	
	for i in range(N):
		data_idx = [j for j in range(i)]
		data_idx.extend([j for j in range(i+1,N)])
		alg.fit(XX[:,data_idx],np.squeeze(XX[:,i]))
		c = np.zeros(N)
		c[data_idx] = alg.coef_
		C[:,i] = c
	maps=SpectralEmbedding(affinity='precomputed', n_components=L, eigen_solver='arpack').fit(np.abs(C)+np.abs(C.T))
  # Se utiliza ahora una descomposición por eigenvectors
	sc = SpectralClustering(n_clusters=L, affinity='precomputed', n_init=50, n_jobs=-1, eigen_solver='arpack')
	sc.fit(np.abs(C) + np.abs(C.T)) # Se introduce la matriz de afinidad descrita en SUB-GAN
	
	return sc.labels_, maps.embedding_

In [80]:
a,b = ssc_mps_modificado(testo,10,20)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\omp.py:657: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [87]:
a.shape


(500,)

In [88]:
b.shape

(500, 20)

In [81]:
data_X = np.load('data_songs.npy')
prev_X = np.load('data_songs_prev.npy')

In [85]:
data_X.shape

(4950, 1, 16, 128)

In [58]:
sample_z = np.random.normal(0, 1, size=(100 , 1))
sample_z.shape

(100, 1)

In [69]:
type((b[1][:,None]))

numpy.ndarray

In [70]:
np.concatenate((b[1][:,None],sample_z))

array([[-0.37352497],
       [ 0.70789136],
       [-0.04358424],
       [ 0.08253972],
       [-0.06485564],
       [-0.26962363],
       [-0.09681702],
       [ 0.26241858],
       [-0.25693877],
       [ 0.59704933],
       [-0.12700554],
       [-0.26304902],
       [-0.06171363],
       [-0.36538481],
       [-0.24948218],
       [-0.99407342],
       [ 4.0375232 ],
       [-0.29798867],
       [ 0.09050725],
       [-0.06521405],
       [-0.02765202],
       [-0.17129503],
       [-0.39039054],
       [ 0.73340963],
       [ 1.29109535],
       [ 0.92420395],
       [-0.32736008],
       [ 0.53937101],
       [ 0.57119776],
       [-1.28082399],
       [-0.99007389],
       [ 0.43124949],
       [ 0.94674216],
       [ 0.29898747],
       [ 1.10548297],
       [ 1.7454993 ],
       [-0.55028412],
       [-1.27830192],
       [-0.07580768],
       [ 1.53896838],
       [ 0.8470109 ],
       [-0.45680263],
       [-0.57182742],
       [-0.06341179],
       [-0.36543548],
       [ 1

In [79]:
k=20
zero = np.repeat(0,k)
zero[k-1]=1
zero

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [45]:
import matplotlib.pyplot as plt
from scipy import stats

In [47]:
stats.describe(b)

DescribeResult(nobs=500, minmax=(array([-0.39964239, -0.40911257, -0.28072443, -0.43176653, -0.4127582 ,
       -0.62786676, -0.33755271, -0.54082473, -0.25693877, -0.88826662,
       -0.89092506, -0.40961708, -0.24553457, -0.36538481, -0.24948218,
       -0.99407342, -1.14125734, -0.96298964, -1.50123788, -0.9187884 ]), array([1.34557425, 0.70789136, 0.48422638, 0.73714752, 0.61040346,
       0.87721982, 0.77344576, 0.98498257, 0.92453899, 1.59916178,
       1.0479362 , 1.29387175, 1.1761867 , 0.9322044 , 0.88076184,
       2.16534818, 4.0375232 , 1.01060642, 2.67590541, 1.25274056])), mean=array([-3.86032769e-03,  1.21622333e-02,  1.24060744e-02,  6.81709111e-03,
        6.66541995e-03,  1.70022136e-03, -2.39355288e-03,  2.21753613e-03,
        9.98588588e-06,  1.21813794e-02,  1.25500915e-03, -4.49148107e-04,
        1.04711890e-02, -7.99940484e-04, -4.69167902e-03, -3.83332255e-16,
       -7.82485188e-16, -3.15664161e-16, -2.81108470e-16, -2.25597319e-16]), variance=array([0.007562

In [44]:
np.maximum(b)

ValueError: invalid number of arguments

In [11]:
np.save('data_songs_prev.npy', data_songs_prev, allow_pickle=True, fix_imports=True)

In [12]:
import os
import scipy.misc
import numpy as np
from model2 import MidiNet #Llamando el modelo Utilizar model2 para eliminar silencios
from utils import pp, to_json, generation_test

# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import tensorflow as tf

flags = tf.app.flags
flags.DEFINE_integer("epoch", 50, "Epoch to train [20]")
flags.DEFINE_float("learning_rate", 0.00005, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_integer("batch_size", 120, "The size of batch [120]")
flags.DEFINE_integer("output_w", 16, "The size of the output segs to produce [16]")
flags.DEFINE_integer("output_h", 128, "The size of the output note to produce [128]")
flags.DEFINE_integer("c_dim", 1, "Number of Midi track. [1]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("sample_dir", "samples", "Dircacectory name to save the image samples [samples]")
flags.DEFINE_string("dataset", "MidiNet_vg", "The name of dataset ")
flags.DEFINE_boolean("is_train", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("is_crop", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("generation_test", False, "True for generation_test, False for nothing [False]")
flags.DEFINE_string("gen_dir", "gen", "Directory name to save the generate samples [samples]")
FLAGS = flags.FLAGS

def main(_):
    pp.pprint(flags.FLAGS.__flags)

    if not os.path.exists(FLAGS.checkpoint_dir):
        os.makedirs(FLAGS.checkpoint_dir)
    if not os.path.exists(FLAGS.sample_dir):
        os.makedirs(FLAGS.sample_dir)
    if not os.path.exists(FLAGS.gen_dir):
        os.makedirs(FLAGS.gen_dir)

    with tf.Session() as sess:
        if FLAGS.dataset == 'MidiNet_vg':
            model = MidiNet(sess, batch_size=FLAGS.batch_size, output_w=FLAGS.output_w, output_h=FLAGS.output_h, c_dim=FLAGS.c_dim,
                    dataset_name=FLAGS.dataset, is_crop=FLAGS.is_crop, checkpoint_dir=FLAGS.checkpoint_dir, sample_dir=FLAGS.sample_dir, 
                            gen_dir=FLAGS.gen_dir)
        
        if FLAGS.is_train:
            model.train(FLAGS)
        else:
            model.load(FLAGS.checkpoint_dir)

        

if __name__ == '__main__':
    tf.app.run()

{'batch_size': <absl.flags._flag.Flag object at 0x000002DD8EB07128>,
 'beta1': <absl.flags._flag.Flag object at 0x000002DD8EBA3550>,
 'c_dim': <absl.flags._flag.Flag object at 0x000002DD8EB07240>,
 'checkpoint_dir': <absl.flags._flag.Flag object at 0x000002DD8EB072E8>,
 'dataset': <absl.flags._flag.Flag object at 0x000002DD8EB07438>,
 'epoch': <absl.flags._flag.Flag object at 0x000002DD8EBA34A8>,
 'gen_dir': <absl.flags._flag.Flag object at 0x000002DD8EB07630>,
 'generation_test': <absl.flags._flag.BooleanFlag object at 0x000002DD8EB07668>,
 'h': <tensorflow.python.platform.app._HelpFlag object at 0x000002DD8EB07748>,
 'help': <tensorflow.python.platform.app._HelpFlag object at 0x000002DD8EB07748>,
 'helpfull': <tensorflow.python.platform.app._HelpfullFlag object at 0x000002DD8EB077F0>,
 'helpshort': <tensorflow.python.platform.app._HelpshortFlag object at 0x000002DD8EB07860>,
 'is_crop': <absl.flags._flag.BooleanFlag object at 0x000002DD8EB075C0>,
 'is_train': <absl.flags._flag.Boolea

Epoch: [ 2] [   3/  41] time: 130.0163, d_loss: 0.95183015, g_loss: 13.94101429
Epoch: [ 2] [   4/  41] time: 130.7603, d_loss: 0.92547536, g_loss: 13.52603054
Epoch: [ 2] [   5/  41] time: 131.5043, d_loss: 0.93081152, g_loss: 14.29981899
Epoch: [ 2] [   6/  41] time: 132.2483, d_loss: 0.90426469, g_loss: 13.83683109
Epoch: [ 2] [   7/  41] time: 132.9903, d_loss: 0.91323638, g_loss: 12.84813023
Epoch: [ 2] [   8/  41] time: 133.7373, d_loss: 0.95659518, g_loss: 12.17816544
Epoch: [ 2] [   9/  41] time: 134.4783, d_loss: 0.89988351, g_loss: 12.39156342
Epoch: [ 2] [  10/  41] time: 135.2194, d_loss: 0.92632294, g_loss: 12.76428986
Epoch: [ 2] [  11/  41] time: 135.9714, d_loss: 0.90177655, g_loss: 12.31659412
Epoch: [ 2] [  12/  41] time: 136.7293, d_loss: 0.94074965, g_loss: 11.34145927
Epoch: [ 2] [  13/  41] time: 137.4763, d_loss: 0.93104029, g_loss: 12.96094894
Epoch: [ 2] [  14/  41] time: 138.2532, d_loss: 0.89593571, g_loss: 12.23542309
Epoch: [ 2] [  15/  41] time: 139.0042, 

Epoch: [ 4] [  23/  41] time: 208.4308, d_loss: 0.66097957, g_loss: 9.07173729
Epoch: [ 4] [  24/  41] time: 209.1838, d_loss: 0.64218014, g_loss: 7.44014788
Epoch: [ 4] [  25/  41] time: 209.9457, d_loss: 0.64935780, g_loss: 8.13487339
Epoch: [ 4] [  26/  41] time: 210.6877, d_loss: 0.64259738, g_loss: 7.98385048
Epoch: [ 4] [  27/  41] time: 211.4298, d_loss: 0.64166576, g_loss: 7.66523409
Epoch: [ 4] [  28/  41] time: 212.1748, d_loss: 0.65168709, g_loss: 8.50971794
Epoch: [ 4] [  29/  41] time: 212.9208, d_loss: 0.62853879, g_loss: 8.31268406
Epoch: [ 4] [  30/  41] time: 213.6658, d_loss: 0.64644605, g_loss: 7.49187374
Epoch: [ 4] [  31/  41] time: 214.4078, d_loss: 0.62520742, g_loss: 8.13017178
Epoch: [ 4] [  32/  41] time: 215.1568, d_loss: 0.62638760, g_loss: 8.09382534
Epoch: [ 4] [  33/  41] time: 215.8988, d_loss: 0.61490208, g_loss: 8.57981300
Epoch: [ 4] [  34/  41] time: 216.6458, d_loss: 0.63194418, g_loss: 8.14366341
Epoch: [ 4] [  35/  41] time: 217.3958, d_loss: 0.60

Epoch: [ 7] [   2/  41] time: 284.6670, d_loss: 0.52522379, g_loss: 8.68574905
Epoch: [ 7] [   3/  41] time: 285.4120, d_loss: 0.55290991, g_loss: 9.07563591
Epoch: [ 7] [   4/  41] time: 286.1580, d_loss: 0.52834028, g_loss: 7.62873745
Epoch: [ 7] [   5/  41] time: 286.9000, d_loss: 0.53573620, g_loss: 8.97181702
Epoch: [ 7] [   6/  41] time: 287.6380, d_loss: 0.51552719, g_loss: 8.54185104
Epoch: [ 7] [   7/  41] time: 288.3880, d_loss: 0.51150757, g_loss: 7.35900831
Epoch: [ 7] [   8/  41] time: 289.1410, d_loss: 0.52254665, g_loss: 7.75626421
Epoch: [ 7] [   9/  41] time: 289.8880, d_loss: 0.50754297, g_loss: 8.14018059
Epoch: [ 7] [  10/  41] time: 290.6330, d_loss: 0.52405214, g_loss: 7.70982695
Epoch: [ 7] [  11/  41] time: 291.3770, d_loss: 0.52338815, g_loss: 7.97783947
Epoch: [ 7] [  12/  41] time: 292.1170, d_loss: 0.54263443, g_loss: 7.13332462
Epoch: [ 7] [  13/  41] time: 292.8601, d_loss: 0.53062898, g_loss: 9.04253006
[Sample] d_loss: 0.51698071, g_loss: 9.14777279
Epoc

Epoch: [ 9] [  22/  41] time: 362.9001, d_loss: 0.49735275, g_loss: 8.40576267
Epoch: [ 9] [  23/  41] time: 363.6930, d_loss: 0.47604078, g_loss: 7.65060854
Epoch: [ 9] [  24/  41] time: 364.4480, d_loss: 0.46706855, g_loss: 6.96890831
Epoch: [ 9] [  25/  41] time: 365.2060, d_loss: 0.49315470, g_loss: 7.38937664
Epoch: [ 9] [  26/  41] time: 365.9619, d_loss: 0.44575667, g_loss: 7.25445747
Epoch: [ 9] [  27/  41] time: 366.7099, d_loss: 0.45513111, g_loss: 6.85888481
Epoch: [ 9] [  28/  41] time: 367.4659, d_loss: 0.48101145, g_loss: 7.24225664
Epoch: [ 9] [  29/  41] time: 368.2109, d_loss: 0.46505973, g_loss: 7.77437878
Epoch: [ 9] [  30/  41] time: 368.9699, d_loss: 0.47127351, g_loss: 6.82483721
Epoch: [ 9] [  31/  41] time: 369.7209, d_loss: 0.44977683, g_loss: 7.57334518
[Sample] d_loss: 0.46314719, g_loss: 8.81254292
Epoch: [ 9] [  32/  41] time: 370.5656, d_loss: 0.48207009, g_loss: 7.50069284
Epoch: [ 9] [  33/  41] time: 371.3316, d_loss: 0.46017608, g_loss: 7.89589024
Epoc

...

Epoch: [44] [   9/  41] time: 1446.1170, d_loss: 0.37913907, g_loss: 6.99581623
Epoch: [44] [  10/  41] time: 1446.8460, d_loss: 0.38003385, g_loss: 7.25057364
Epoch: [44] [  11/  41] time: 1449.7183, d_loss: 0.36762702, g_loss: 7.81783295
Epoch: [44] [  12/  41] time: 1450.5800, d_loss: 0.38641995, g_loss: 6.81086588
Epoch: [44] [  13/  41] time: 1451.3859, d_loss: 0.38230523, g_loss: 7.41525030
Epoch: [44] [  14/  41] time: 1452.2117, d_loss: 0.37534037, g_loss: 7.43281412
Epoch: [44] [  15/  41] time: 1453.0903, d_loss: 0.37881038, g_loss: 7.32309961
Epoch: [44] [  16/  41] time: 1453.8742, d_loss: 0.37241662, g_loss: 7.76259661
Epoch: [44] [  17/  41] time: 1454.6551, d_loss: 0.38406202, g_loss: 7.87032461
Epoch: [44] [  18/  41] time: 1455.4191, d_loss: 0.36594930, g_loss: 7.42547369
Epoch: [44] [  19/  41] time: 1456.2000, d_loss: 0.36710027, g_loss: 7.87552261
Epoch: [44] [  20/  41] time: 1456.9720, d_loss: 0.37770060, g_loss: 8.27163506
Epoch: [44] [  21/  41] time: 1457.7359,

Epoch: [46] [  28/  41] time: 1524.5132, d_loss: 0.36562291, g_loss: 7.49677038
Epoch: [46] [  29/  41] time: 1525.2433, d_loss: 0.37248993, g_loss: 8.13020420
Epoch: [46] [  30/  41] time: 1525.9863, d_loss: 0.37482563, g_loss: 7.03978872
Epoch: [46] [  31/  41] time: 1526.7084, d_loss: 0.38310868, g_loss: 7.05420303
Epoch: [46] [  32/  41] time: 1527.4324, d_loss: 0.38762316, g_loss: 7.29964352
Epoch: [46] [  33/  41] time: 1528.1575, d_loss: 0.36807021, g_loss: 7.21909428
Epoch: [46] [  34/  41] time: 1528.8796, d_loss: 0.38472289, g_loss: 7.12723923
Epoch: [46] [  35/  41] time: 1529.6056, d_loss: 0.38035864, g_loss: 7.59136438
Epoch: [46] [  36/  41] time: 1530.3596, d_loss: 0.38612482, g_loss: 7.03191185
Epoch: [46] [  37/  41] time: 1531.1146, d_loss: 0.38334996, g_loss: 7.76369810
Epoch: [46] [  38/  41] time: 1531.8945, d_loss: 0.38626409, g_loss: 7.19425201
Epoch: [46] [  39/  41] time: 1532.6395, d_loss: 0.37314227, g_loss: 7.28639650
Epoch: [46] [  40/  41] time: 1533.3925,

Epoch: [49] [   6/  41] time: 1601.0601, d_loss: 0.38657701, g_loss: 7.70417738
Epoch: [49] [   7/  41] time: 1601.8899, d_loss: 0.38039950, g_loss: 7.07582140
Epoch: [49] [   8/  41] time: 1602.6359, d_loss: 0.37528309, g_loss: 7.74662733
Epoch: [49] [   9/  41] time: 1603.4377, d_loss: 0.37967300, g_loss: 7.27720213
Epoch: [49] [  10/  41] time: 1604.2077, d_loss: 0.37816542, g_loss: 7.62495708
Epoch: [49] [  11/  41] time: 1604.9297, d_loss: 0.36860725, g_loss: 8.08406544
Epoch: [49] [  12/  41] time: 1605.6648, d_loss: 0.37909490, g_loss: 6.93829489
Epoch: [49] [  13/  41] time: 1606.3859, d_loss: 0.39375114, g_loss: 7.68360329
Epoch: [49] [  14/  41] time: 1607.1668, d_loss: 0.38540149, g_loss: 7.35290051
Epoch: [49] [  15/  41] time: 1607.8948, d_loss: 0.38488227, g_loss: 7.51964521
Epoch: [49] [  16/  41] time: 1608.6448, d_loss: 0.36239672, g_loss: 8.21738338
Epoch: [49] [  17/  41] time: 1609.3759, d_loss: 0.37877604, g_loss: 8.37757397
Epoch: [49] [  18/  41] time: 1610.1358,

SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!tensorboard --logdir /logs

In [ ]:
# !pip install music21

In [23]:
folder_png ='C:/Users/Gustavo/Documents/GitHub/GAN_project_PUCP/Midinet_VG/Model_type1/samples'
import os
import numpy as np
import matplotlib.pyplot as plt
import music21

In [24]:
def generate_midis(png_dir):
    for f in os.listdir(png_dir):    
        image_plt = plt.imread(os.path.join(png_dir,f))
        image_arr = np.array(image_plt)
        r_song = []
        for i in range(0,640,128):
            print(i)
            r_measure = []
            for j in range(0,16):
                measure = image_arr[j][i:i+128]
                #print(len(measure))
                r_measure.append([0 if x[0]<0.65 else x[0] for x in measure])
            r_song.append(r_measure)
        new_test_stream = music21.stream.Stream()
        sup = r_song
        new_track = music21.stream.Part()
        for j in range(0,len(sup)):
            measure = sup[j]
            for k in range(0,len(measure)):
                if sum(measure[k]) == 0 :
                    new_track.append(music21.note.Rest(type='16th'))
                else:
                    note_value = np.argmax(measure[k])
                    new_track.append(music21.note.Note(note_value, type='16th'))
        new_test_stream.append(new_track)
        new_test_stream.write('midi',f.split('.png')[0]+'.mid')
        new_test_stream.show('midi') 

In [25]:
generate_midis(folder_png)

0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


...

0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512


0
128
256
384
512
